# Twitter analysis

We have a collection of tweets and want to analyse them. Hashtags, influencer, ...

1. Load the JSON file at `/data/twitter/twitter_schueler_10k.json`. Every JSON object is stored within a single line. Thus, add `lines=True` to the `read_json` method.

2. Use `info()` and `describe()` to get an initial understanding of the data

3. When was the first tweet? 

4. Extract tags and store in new column `tags` as list
```
tag_extractor = lambda x: [i for i in x.split() if i.startswith("#")]
```

5. Get a list of all tags
```
def collect_tags(dframe):
    tag_list = []
    for row in df.itertuples():
        if len(row.tags) > 0:
            tag_list.extend(row.tags)
    return tag_list
```

6. Get the top 10 tags and show as pie chart

7. Get the user `user_name` with the most followers `user_followerscount`. Users can appear multiple times, thus, take max of followers. Show the results as a bar chart.

8. Tweets' sentiment. Convert tweet's `text` to  `TextBlob`, get its `sentiment` and then the `polarity`:
```
from textblob import TextBlob
sentiment = lambda x: TextBlob(x).sentiment.polarity
```
Draw a histogram over the `polarity` using `hist(bins=20)`



In [ ]:
# 0. import pandas
import pandas as pd

In [ ]:
# 1. load data
df = pd.read_json("twitter_schueler_10k.json",lines=True)
# 2. info shows column-wise information
df.info()

In [ ]:
# 2. describe shows information about numeric columns
df.describe()

In [ ]:
# 3. the created_at field stores the date and time the tweet was made. Use describe to get min value
print(df["created_at"].describe())

# alternativeley, we can use min
firstTweetDate = df["created_at"].min()
print(f"First tweet in our data was made on: {firstTweetDate}")

In [ ]:
# 4. define the extractor lambda function
tag_extractor = lambda x: [i for i in x.split() if i.startswith("#")]

# apply the extactor function and store tags in new column
df["tags"] = df["text"].map(tag_extractor)
df.head()

In [ ]:
# 5. collect all tags in a single list 
def collect_tags(dframe):
    tag_list = []
    for row in df.itertuples():
        if len(row.tags) > 0:
            tag_list.extend(row.tags)
    return tag_list

all_tags = collect_tags(df)
print(len(all_tags))
all_tags_set = set(all_tags) # unique tags
print(len(all_tags_set))

In [ ]:
# 6. to get top ten, group by tag and count number of elements in each group
tagDF = pd.DataFrame(all_tags, columns=["tag"])
tagCount = tagDF.groupby(["tag"]).size().reset_index(name="count")
tagCountSorted = tagCount.sort_values(["count"],ascending=False)
topTen = tagCountSorted[:10]
topTen

In [ ]:
# 6. plot as pie chart
%matplotlib inline
topTen.set_index('tag').plot(y='count', kind='pie', figsize=(10, 10))

In [ ]:
# 7 user with most followers

# group by user_name and calculate max over the user_followerscount column
maxFollower = df.groupby(['user_name'], as_index=False)['user_followerscount'].max() 
maxFollower = maxFollower.sort_values("user_followerscount", ascending=False)
maxFollower

In [ ]:
# maxFollowers has more than 6000 rows... we just want to plot the first 10 
maxFollower.head(10).set_index('user_name').plot(y="user_followerscount", kind="bar")

In [ ]:
from textblob import TextBlob # requires textblob module to be installed, e.g.:  pip3 install -U --user textblob
sentiment = lambda x: TextBlob(x).sentiment.polarity

df["sentiment"] = df["text"].map(sentiment)
df["sentiment"].hist(bins=20)